In [1]:
# importing required libraries

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.metrics import r2_score

In [2]:
train_data= pd.read_csv('train_BRCpofr.csv',index_col=0)
test_data= pd.read_csv('test.csv',index_col=0)

In [3]:
train_data.loc[train_data.income =='<=2L','income']= 'less than 2L'
test_data.loc[test_data.income =='<=2L','income']= 'less than 2L'

In [4]:
train_data.head(5)

,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy,cltv
id,,,,,,,,,,,
1,Male,Urban,Bachelor,5L-10L,1,5,5790,More than 1,A,Platinum,64308
2,Male,Rural,High School,5L-10L,0,8,5080,More than 1,A,Platinum,515400
3,Male,Urban,Bachelor,5L-10L,1,8,2599,More than 1,A,Platinum,64212
4,Female,Rural,High School,5L-10L,0,7,0,More than 1,A,Platinum,97920
5,Male,Urban,High School,More than 10L,1,6,3508,More than 1,A,Gold,59736


In [5]:
train_data.marital_status= train_data.marital_status.astype('str')
test_data.marital_status= test_data.marital_status.astype('str')

In [6]:
# Removing duplicates
train_data = train_data.drop_duplicates()

In [7]:
y = train_data.pop('cltv')

In [8]:
categorical_columns= train_data.select_dtypes('object').columns

In [9]:
X_train, X_test, y_train, y_test = train_test_split(train_data, y, test_size=0.25, random_state=22)

In [10]:
from catboost import CatBoostRegressor
import numpy as np


# initialize the model
cat_model = CatBoostRegressor(iterations=1400, learning_rate=0.01, depth=5,\
                          loss_function='RMSE', cat_features=list(categorical_columns))

# fit the model on the training data
cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), logging_level='Silent',plot=True)

# make predictions on the test set
y_pred = cat_model.predict(X_test)

# evaluate the model

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: ", rmse)

r2_score(y_test, y_pred)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

RMSE:  82213.62074182871


0.1653480364856783

In [11]:
# seeing important features
feature_importance= pd.DataFrame({'feature':X_train.columns,'importance':cat_model.feature_importances_})
feature_importance.sort_values(by='importance',ascending=False)[:10].reset_index(drop=['id'])

,feature,importance
0,num_policies,80.850248
1,claim_amount,7.883702
2,policy,3.348487
3,area,3.186376
4,marital_status,1.313141
5,income,1.249338
6,qualification,0.767325
7,gender,0.570694
8,type_of_policy,0.476525
9,vintage,0.354164


Num of policies and Claim amount as major factor in prediction was also predicted during data exploration.

In [12]:
prediction= cat_model.predict(test_data)

submission = pd.DataFrame({'id':test_data.index,'cltv':prediction})

submission.to_csv('final_submission.csv',index=False)